##### Task 1 (8 Points)

In [48]:
"""
    topic: Paths in triangles

    a)
    Implement an algorithm that constructs such a triangle with a depth of d and stores it as a list of length n.
    choose numbers from 0, ..., 99 for the entries

    b) 
    Implement an algorithm with a runtime of O(n) that calculates the shortest path in a triangular structure. 
    In this context, the length of a path is determined by summing the values of all visited points in the triangle. 
    Movement options are limited to going up left or up right from a given point within the triangle.

"""
import numpy as np
import math

def build_triangle(depth):
    number = np.arange(1, depth + 2)
    return np.random.randint(0, 100, size=np.sum(number))

def min_path(depth):
    print("We generate a triangle in form a list with random values")
    triangle = build_triangle(depth=depth)
    print(triangle)
    print("\n")
    
    print("Here we initialize a matrix with zeros for n x n")
    print("(n = depth + 1)")
    triangle_array = np.zeros((depth + 1, depth + 1))
    print(triangle_array)
    print("\n")

    print("np.tri(...) is a useful method for generating a matrix with 1's at and below the diagonal and zeros everywhere else")
    mask = np.tri(depth + 1,dtype=bool)
    print(mask)
    print("\n")

    print("we use the mask to assign the values from the generated triangle in the beginning")
    triangle_array[mask] = triangle
    print(triangle_array)
    print("\n")
    
    for i in range(depth - 1, -1, -1):
        for j in range(i + 1):
            triangle_array[i, j] += min(triangle_array[i + 1, j], triangle_array[i + 1, j + 1])
        print(triangle_array)
        print("\n")

    return triangle_array[0, 0]


In [49]:
min_path(2)

We generate a triangle in form a list with random values
[20 37 85 10  9 42]


Here we initialize a matrix with zeros for n x n
(n = depth + 1)
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


np.tri(...) is a useful method for generating a matrix with 1's at and below the diagonal and zeros everywhere else
[[ True False False]
 [ True  True False]
 [ True  True  True]]


we use the mask to assign the values from the generated triangle in the beginning
[[20.  0.  0.]
 [37. 85.  0.]
 [10.  9. 42.]]


[[20.  0.  0.]
 [46. 94.  0.]
 [10.  9. 42.]]


[[66.  0.  0.]
 [46. 94.  0.]
 [10.  9. 42.]]




66.0

##### Task 2 (12 Points)

In [106]:
""" 
    topic: graphs

    a)
    Implement and utilize a Graph class for constructing a graph. 
    The class should include functionality for adding edges both through the add_edge() method and from a file named graph.txt. 
    The file format is expected to have the number of vertices in the first row, followed by edge information below it.

    b) 
    Implement the number_of_neighbors() function within the Node class.

    c)
    Implement the is_simple() function  within Graph class.

    d) 
    Implement the adjancency_matrix() function within Graph class.
    Store the adjacency matrix as an ndarray.
"""


class Graph:
    invalid_node = -1

    def __init__(self, numorfile, directed = False):
        self._directed = directed
        if type(numorfile) == int:
            self._nodes = [self.Node() for i in range(numorfile)]
        elif type(numorfile)  == str:
            with open(numorfile, "r") as file:
                numnodes = int(file.readline())
                self._nodes = [self.Node() for i in range(numnodes)]
                for line in file:
                    tail = int(line.split()[0])
                    head = int(line.split()[1])
                    if tail != head:
                        self.add_edge(tail, head)
                    else:
                        raise RuntimeError("Invalid file format: loops not allowed")
        else:
            raise NotImplementedError("Type of Argument not allowed")

    def num_nodes(self):
        return len(self._nodes)

    def add_nodes(self, num_new_nodes):
        self._nodes.extend([self.Node() for i in range(num_new_nodes)])

    def add_edge(self, tail, head):
        if (tail >= self.num_nodes() or tail < 0 or head >= self.num_nodes() or head <0):
            raise ValueError("Edge cannot be added due to undefined endpoint")
        self._nodes[tail].add_neighbor(head)
        if not self._directed:
           self._nodes[head].add_neighbor(tail)

    def get_node(self, nodeId):
        if (nodeId <0 or nodeId >= self.num_nodes()):
            raise ValueError("Invalid nodeId")
        return self._nodes[nodeId]

    def __str__(self): #for printing
        out = ""
        if self._directed:
            out +="Digraph "
        else:
            out +="Undirected Graph "
        out +="with {} vertices, numbered 0, ..., {}\n".format(self.num_nodes(), self.num_nodes()-1)
        for nodeId in range(self.num_nodes()):
            if self._directed:
                s = "leaving"
            else:
                s = "incident to"
            out +="The following edges are "+ s +" vertex {}:\n".format(nodeId)
            for neighbor in self._nodes[nodeId].adjacent_nodes():
                out +="{} - {}\n".format(nodeId, neighbor.id())
        return out
    ############### c ###############
    def is_simple(self):
        edges = [element for element in self.__str__().split("\n")[1:-1] if "-" in element]
        return len(set(edges)) == len(edges)
    ############### d ###############
    def adjancency_matrix(self):
        num_nodes = self.num_nodes()
        matrix = [[0] * num_nodes for _ in range(num_nodes)]

        for nodeId in range(num_nodes):
            neighbors = self._nodes[nodeId].adjacent_nodes()
            for neighbor in neighbors:
                matrix[nodeId][neighbor.id()] = 1
        return np.array(matrix)


    class Node:
        def __init__(self):
            self._neighbors = []

        def add_neighbor(self, nodeId):
            self._neighbors.append(Graph.Neighbor(nodeId))

        def adjacent_nodes(self):
            return self._neighbors
        ############### b ###############
        def number_of_neighbors(self):
            return len(self._neighbors)

    class Neighbor:
        def __init__(self, nodeId):
            self._id = nodeId

        def id(self):
            return self._id


In [59]:
############### a ###############
file = "./graph.txt"
Graph(file,directed=True)

In [107]:
g = Graph(6,directed=True)

In [108]:
g.add_edge(0,1)
g.add_edge(1,5)
g.add_edge(5,0)
g.add_edge(4,0)
g.add_edge(3,4)

In [111]:
g.adjancency_matrix()

array([[0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])